# Custom waveform signal generation

In this example, we will learn how to create a custom signal using Red Pitaya as an arbitrary waveform generator.

## Libraries and FPGA image
In addition to the usual libraries, we will also import *numpy* for easier array operations, as well as *matplotlib* for signal plotting.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from rp_overlay import overlay
import rp

fpga = overlay()
rp.rp_Init()

## Macros
Throughout this tutorial we will mention macros multiple times. Here is a complete list of macros that will come in handy when customising this notebook. The marcos are a part of the **rp** library.

- **Waveforms** - RP_WAVEFORM_SINE, RP_WAVEFORM_SQUARE, RP_WAVEFORM_TRIANGLE, RP_WAVEFORM_RAMP_UP, RP_WAVEFORM_RAMP_DOWN, RP_WAVEFORM_DC, RP_WAVEFORM_PWM, RP_WAVEFORM_ARBITRARY, RP_WAVEFORM_DC_NEG, RP_WAVEFORM_SWEEP
- **Generator modes** - RP_GEN_MODE_CONTINUOUS, RP_GEN_MODE_BURST
- **Sweep direction** - RP_GEN_SWEEP_DIR_NORMAL, RP_GEN_SWEEP_DIR_UP_DOWN
- **Sweep mode** - RP_GEN_SWEEP_MODE_LINEAR, RP_GEN_SWEEP_MODE_LOG
- **Generator trigger source** - RP_GEN_TRIG_SRC_INTERNAL, RP_GEN_TRIG_SRC_EXT_PE, RP_GEN_TRIG_SRC_EXT_NE
- **Fast analog channels** - RP_CH_1, RP_CH_2
- **Fast analog triggers** - RP_T_CH_1, RP_T_CH_2, RP_T_CH_EXT
- **Rise and fall times** - RISE_FALL_MIN_RATIO, RISE_FALL_MAX_RATIO

SIGNALlab 250-12 only:
- **Generator gain** - RP_GAIN_1X, RP_GAIN_5X

STEMlab 125-14 4-Input only:
- **Fast analog channels** - RP_CH_3, RP_CH_4
- **Fast analog triggers** - RP_T_CH_3, RP_T_CH_4

## Prepare an arbitrary signal
Let us define our custom signal as a sum of two sine waves with different frequencies. We will create an array with one period of custom signal that Red Pitaya will store inside an FPGA buffer, which is used it to generate the custom signal.

Before we jump into coding, here is some theory.

Red Pitaya expects a 16384 sample-long array, which is then stored in an internal FPGA buffer. There are a few things worth mentioning here:
- Red Pitaya expects and will act as the whole buffer containing only one** period of the signal. Consequently, settings like frequency apply to the full buffer of 16384 samples.
- If multiple signal periods are defined within the buffer, the output signal will generate faster than the specified frequency setting. For example, defining two sine periods inside the buffer and setting the frequency to 10 kHz will result in a 20 kHz sine output.
- Sending a shorter array/buffer also results in a higher output frequency. For example, defining one sine period with 8192 samples and setting the frequency to 10 kHz results in a 20 kHz sine output.
- The expected signal range is +-1 V. Signals larger than +-1 V will result in automatic normalisation, but smaller signals will not be normalised.

In [ ]:
channel = rp.RP_CH_1        # rp.RP_CH_2
channel2 = rp.RP_CH_2
waveform = rp.RP_WAVEFORM_ARBITRARY
freq = 10000
ampl = 1

N = 16384       # Number of samples in the buffer

##### Custom waveform setup #####
x = rp.arbBuffer(N)                             # Defining buffers
y = rp.arbBuffer(N)

t = np.linspace(0, 1, N)*2*np.pi                # "time axis"

x_temp = np.sin(t) + 1/3*np.sin(3*t)            # First custom signal
y_temp = 1/2*np.sin(t) + 1/4*np.sin(4*t)        # Second custom signal

for i in range(0, N, 1):
    x[i] = float(x_temp[i])                     # Copying signals to buffers
    y[i] = float(y_temp[i])

# Plot both signals
plt.plot(t, x, 'r')
plt.plot(t, y, 'b')
plt.xlabel('N')
plt.ylabel('V')
plt.title('Signals')
plt.show()

The first custom signal is a combination of a standard sine signal and the third harmonic frequency component.
The second signal is a combination of sine and the forth harmonic component.

Notice that during the custom signal definition we did not use the actual frequency, but rather defined the sine waves relatively to each other. The actual output frequency is defined with Red Pitaya SCPI commands.

In [ ]:
# Reset generator
rp.rp_GenReset()

###### Generation #####
rp.rp_GenWaveform(channel, waveform)
rp.rp_GenArbWaveform(channel, x.cast(), N)      # Defining the custom signal (writing to the FPGA buffer)
rp.rp_GenFreqDirect(channel, freq)
rp.rp_GenAmp(channel, ampl)

rp.rp_GenWaveform(channel2, waveform)
rp.rp_GenArbWaveform(channel2, y.cast(), N)     # Defining the custom signal (writing to the FPGA buffer)
rp.rp_GenFreqDirect(channel2, freq)
rp.rp_GenAmp(channel2, ampl)

# Enable output synchronisation
rp.rp_GenOutEnableSync(True)

# Sync trigger output channels
rp.rp_GenSynchronise()


# Release resources
rp.rp_Release()

### Multiple periods inside the AWG buffer
If there are multiple periods inside the arbitrary waveform generator buffer, use the following equation to convert the frequency set with *rp_GenFreqDirect()* to the Red Pitaya output frequency.

> $f_{IN} = \frac{f_{OUT, TARGET} \cdot N_{PERIOD}}{N_{BUFF}}$

Where:
- $f_{IN}$ - frequency set with *rp_GenFreqDirect()*
- $f_{OUT, TARGET}$ - target output frequency generated by Red Pitaya
- $N_{PERIOD}$ - number of samples in one signal period inside the buffer/array
- $N_{BUFF}$ - length of the whole AWG array/buffer (the expected length is 16384)


### Note
There are a lot of different commands for the Generation. The list of available functions is quite an achievement to read through, so from now on, please refer to the *C and Python API section* of the [Command table](https://redpitaya.readthedocs.io/en/latest/appsFeatures/remoteControl/command_list.html#list-of-supported-scpi-api-commands) for all available commands.
